# Summorize Pharma COUNT and create xlsx-Files

In [1]:
import pandas as pd
import mysql.connector
import sys
sys.path.insert(0, '../../data/lib/')
import consts
import mysql.connector

In [2]:
year = 2019

## Load DB

In [3]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

df_recipient = pd.read_sql("""SELECT
  rec_id as id,
  rec_name as name,
  rec_address as address,
  rec_location as location,
  rec_type as type,
  sum(tra_value) as value,
  pha_name as pharma
 FROM transaction
  LEFT JOIN transaction_category ON tra_fk_transaction_category = transaction_category.trc_id
  LEFT JOIN recipient ON tra_fk_recipient = recipient.rec_id
  LEFT JOIN pharma ON tra_fk_pharma = pharma.pha_id
  WHERE tra_year = %s
  GROUP BY tra_fk_recipient, tra_fk_pharma
  ORDER BY rec_name DESC""" % year, con=db_connection)

In [4]:
df_db = df_recipient.groupby(['pharma', 'type']).agg({'name': 'count', 'value': 'sum'})
df_db.head()

name      value
pharma         type                 
 Nordic Pharma hco      1     2340.0
               hcp      1      638.0
A. Menarini AG hco    113  1388257.0
               hcp    222   229002.0
AbbVie         hco    148  5728593.0

## Load Count

In [5]:
df_acc = pd.read_csv('../../data/3. transformation/0_accumulations_%s.csv' % year, encoding='UTF-8')

In [6]:
df_acc.head()

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,NaN,NaN,0.0,0.0,0.0,0.0,0.0,abbvie
1,hco_amount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,abbvie
2,hcp_count,NaN,NaN,0.0,0.0,0.0,0.0,0.0,abbvie
3,hco_count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,abbvie
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,1451064.0,abbvie


In [10]:
df_acc_count = df_acc[df_acc.type != 'rnd'].copy()
df_acc_count['category'] = df_acc_count['type'].apply(lambda x: x)
df_acc_count['type'] = df_acc_count['type'].apply(lambda x: 'hcp' if x in ['hcp_amount', 'hcp_count'] else 'hco')

df_acc_count.groupby(['source', 'type']).count().head()

#df_agg_count = df_acc[df_acc.type.isin(['hcp_count', 'hco_count'])].copy()
#df_agg_count = df_agg_count.groupby(['source', 'type']).agg({'total': 'sum'}).reset_index()
#df_agg_count = df_agg_count.rename(columns={'total': 'name'})
#df_agg_count.head()

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source,category
0,hcp,NaN,NaN,0.0,0.00,0.00,0.00,0.00,abbvie,hcp_amount
1,hco,0.0,0.00,0.0,0.00,0.00,0.00,0.00,abbvie,hco_amount
2,hcp,NaN,NaN,0.0,0.00,0.00,0.00,0.00,abbvie,hcp_count
3,hco,0.0,0.00,0.0,0.00,0.00,0.00,0.00,abbvie,hco_count
5,hcp,NaN,NaN,6077.0,7021.00,3000.00,0.00,16098.00,allergan,hcp_amount
6,hcp,NaN,NaN,5.0,5.00,1.00,0.00,11.00,allergan,hcp_count
7,hco,2220.0,NaN,NaN,NaN,NaN,NaN,2220.00,allergan,hco_amount
8,hco,2.0,0.00,NaN,NaN,0.00,NaN,2.00,allergan,hco_count
10,hcp,NaN,NaN,NaN,12237.72,31156.65,780.07,44174.44,almirall,hcp_amount
11,hco,NaN,39235.57,NaN,NaN,3000.00,NaN,42235.57,almirall,hco_amount


In [ ]:
df_rec_count = df_rec.groupby('source').agg({'name': 'count'}).reset_index()
df_rec_count.head()

## Merge

In [ ]:
df_count = pd.concat([df_agg_count, df_rec_count])
df_count = df_count.groupby('source').agg({'name': 'sum'})

## Get Sum

In [ ]:
df_acc_sum = df_acc[df_acc.type.isin(['hcp_amount', 'hco_amount', 'rnd'])].copy()
df_acc_sum = df_acc_sum.groupby('source').agg({'total': 'sum'})

df_rec_sum = df_rec.groupby('source').agg({'total': 'sum'})

df_sum = pd.concat([df_acc_sum, df_rec_sum])
df_sum = df_sum.groupby('source').agg({'total': 'sum'}).reset_index()

## Join

In [ ]:
df_joined = df_count.merge(df_sum, how='left', on='source')
df_joined = df_joined.rename(columns={'name': 'empfaenger', 'total': 'summe_zahlungen'})
df_joined['durchschnitt_pro_empfaenger'] = round(df_joined['summe_zahlungen'] / df_joined['empfaenger'])

df_joined = df_joined.sort_values('durchschnitt_pro_empfaenger', ascending=False)
df_joined.head()

In [ ]:
#Write to Excel
writer = pd.ExcelWriter('../../data/99. analyzes/excel/pharmas/anzahl_empfaenger.xlsx', options={'encoding':'utf-8'})

df_joined.to_excel(writer, 'Empfänger', index=True)

writer.save()

In [ ]:
df_rec.head()

In [ ]:
-- ALT

## Connect

In [ ]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [ ]:
"""
df_list_raw = pd.read_sql("""SELECT 
    count( DISTINCT tra_fk_recipient) as count,
    pha_name as pharma,
    pha_id as pharmaid,
    tra_year as year
  FROM transaction
  LEFT JOIN pharma ON tra_fk_pharma = pha_id
  LEFT JOIN transaction_category on tra_fk_transaction_category = trc_id
  LEFT JOIN recipient ON rec_id = tra_fk_recipient
  WHERE tra_year = 2019
  GROUP BY tra_fk_pharma
  """, con=db_connection)
"""

In [ ]:
df_list_raw.head(20)

## Export

In [ ]:
def export_year(df_data, year):
    print(year)
    
    if year == 'all':
        df_part = df_data
    else:
        df_part = df_data[df_data.year == year]
    
    #Generate total
    df_pivot_total = df_part.pivot_table(index=['pharma', 'type'], columns='category', values='sum_value', fill_value=0, aggfunc='sum', margins=True, margins_name='total')
    df_pivot_total = df_pivot_total.fillna(0)
    df_pivot_total = df_pivot_total[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'rnd', 'total']]
    df_pivot_total = df_pivot_total.apply(lambda x: round(x, 2), axis=1)  
    
    #Generate detailed
    """
    df_pivot_detail = df_part.pivot_table(index=['pharma', 'declaration'], columns='category', values='sum_value', aggfunc='sum', margins=True, margins_name='total')
    df_pivot_detail = df_pivot_detail.fillna(0)
    df_pivot_detail = df_pivot_detail[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'rnd', 'total']]
    df_pivot_detail = df_pivot_detail.apply(lambda x: round(x, 2), axis=1)
    """
    #Write to Excel
    writer = pd.ExcelWriter('../../data/99. analyzes/excel/pharmas/nach_hco_hcp_rnd_%s.xlsx' % year, options={'encoding':'utf-8'})

    df_pivot_total.to_excel(writer, 'total %s' % year, index=True)

    writer.save()
    
    df_pivot_total.head()

In [ ]:
df_list = df_list_raw.copy()
df_accu = df_accu_raw.copy()

#Concat
df_data = pd.concat([df_list, df_accu], sort=False)

#Export all
export_year(df_data, 'all')

#Export years
for year in df_data['year'].unique():
    export_year(df_data, year)